# 한글 Parent-Child Chunk 를 OpenSearch 에 저장

이 노트북은 한글 전체 문서를 Parent, Child 문서로 구분을 하여 오픈 서치 벡터스토어에 저장을 하고, 테스트를 하는 노트북 입니다. 또한 일부 문서로 검증 인덱스를 생성하여 테스트 용으로 만들기 위한 노트북 입니다.

---

## [중요] 사전 실행 노트북
이 노트북은 아래 두개의 셋업 노트북이 먼저 실행이 되어야 합니다.
- (1) Setup 노트북
    - 경로는 aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/00_setup/setup.ipynb 와 같습니다.
    -  [Setup Notebook](https://github.com/aws-samples/aws-ai-ml-workshop-kr/blob/master/genai/aws-gen-ai-kr/00_setup/setup.ipynb)
- (2) Amazon OpenSearch 설치 노트북    
    - 경로는 aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/00_setup/setup_opensearch.ipynb 와 같습니다.
    - [Setup OpenSearch](https://github.com/aws-samples/aws-ai-ml-workshop-kr/blob/master/genai/aws-gen-ai-kr/00_setup/setup_opensearch.ipynb)

# 1. Bedrock Client 생성

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = ".."
add_python_path(module_path)


python path: /home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot is added
sys.path:  ['/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot/01_preprocess_docs', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages', '/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot']


In [2]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from local_utils import bedrock, print_ww
from local_utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


# 2. Embedding 모델 로딩

## Embedding Model 선택

In [3]:
Use_Titan_Embedding = True
Use_Cohere_English_Embedding = False

## Embedding Model 로딩

In [4]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

if Use_Titan_Embedding:
    llm_emb = BedrockEmbeddings(client=boto3_bedrock, model_id = "amazon.titan-embed-text-v1")
    dimension = 1536
elif Use_Cohere_English_Embedding:
    llm_emb = BedrockEmbeddings(client=boto3_bedrock, model_id = "cohere.embed-english-v3")    
    dimension = 1024
else:
    lim_emb = None

llm_emb

BedrockEmbeddings(client=<botocore.client.BedrockRuntime object at 0x7f9bf917dd80>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-embed-text-v1', model_kwargs=None, endpoint_url=None, normalize=False)

# 3. Load all Json files

In [5]:
from local_utils.proc_docs import get_load_json, show_doc_json

In [6]:
import glob

# Specify the directory and file pattern for .txt files
folder_path = 'data/poc/preprocessed_json/*ko.json'

# List all .txt files in the specified folder
json_files = glob.glob(folder_path)
# json_files = ['data/poc/customer_EFOTA.json']

# Load each item per json file and append to a list
doc_json_list = []
for file_path in json_files:
    doc_json = get_load_json(file_path)
    doc_json_list.append(doc_json)

print("all json files: ", len(doc_json_list))    
# Flatten the list of lists into a single list
all_docs = []
for item in doc_json_list:
        all_docs.extend(item)
        
print("all items: ", len(all_docs))

.[]
all json files:  1
all items:  1732


# 4. Index 생성

## Index 이름 결정

In [7]:
index_name = "v26-genai-poc-knox-kor-parent-doc-retriever"

## Index 스키마 정의

In [8]:
index_body = {
    'settings': {
        'analysis': {'analyzer': {'my_analyzer': {'char_filter': ['html_strip'],
                                                    'tokenizer': 'nori',
                                                       'filter': [
                                                                   'my_nori_part_of_speech'
                                                                 ],
                                                       'type': 'custom'}},
                                   'tokenizer': {'nori': {
                                                  'decompound_mode': 'mixed',
                                                  'discard_punctuation': 'true',
                                                  'type': 'nori_tokenizer'}
                                                },
                                    "filter": {
                                          "my_nori_part_of_speech": {
                                                "type": "nori_part_of_speech",
                                                "stoptags": [
                                                    "J", "XSV", "E", "IC","MAJ","NNB",
                                                    "SP", "SSC", "SSO",
                                                    "SC","SE","XSN","XSV",
                                                    "UNA","NA","VCP","VSV",
                                                    "VX"
                                                ]
                                                
                                          }
                                    }
                    },        
        'index': {
            'knn': True,
            'knn.space_type': 'cosinesimil'  # Example space type
        }
    },
    'mappings': {
        'properties': {
            'metadata': {
                'properties': {
                               'source' : {'type': 'keyword'},                    
                               'last_updated': {'type': 'date'},
                               'project': {'type': 'keyword'},
                               'seq_num': {'type': 'long'},
                               'title': {'type': 'text'},  # For full-text search
                               'url': {'type': 'text'},  # For full-text search
                            }
            },            
            'text': {
                'analyzer': 'my_analyzer',
                'search_analyzer': 'my_analyzer',
                'type': 'text'
            },
            'vector_field': {
                'type': 'knn_vector',
                'dimension': 1536  # Replace with your vector dimension
            }
        }
    }
}


# 5. LangChain OpenSearch VectorStore 생성 
## 선수 조건


## 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.

In [9]:
from local_utils.proc_docs import get_parameter

In [10]:
import boto3
ssm = boto3.client('ssm', 'us-east-1')

opensearch_domain_endpoint = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_userid',
)

opensearch_user_password = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_password',
)


In [11]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

## OpenSearch Client 생성

In [12]:
from local_utils.opensearch import opensearch_utils

In [13]:
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 오픈 서치 인덱스 생성 
- 오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [15]:
from local_utils.opensearch import opensearch_utils

In [16]:

index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        index_name
    )

opensearch_utils.create_index(os_client, index_name, index_body)
index_info = os_client.indices.get(index=index_name)
print("Index is created")
pprint(index_info)

index_name=v26-genai-poc-knox-kor-parent-doc-retriever, exists=False

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'v26-genai-poc-knox-kor-parent-doc-retriever'}
Index is created
{'v26-genai-poc-knox-kor-parent-doc-retriever': {'aliases': {},
                                                 'mappings': {'properties': {'metadata': {'properties': {'last_updated': {'type': 'date'},
                                                                                                         'project': {'type': 'keyword'},
                                                                                                         'seq_num': {'type': 'long'},
                                                                                                         'source': {'type': 'keyword'},
                                                                                                         'title': {'type': 'text'},
                                               

## 랭체인 인덱스 연결 오브젝트 생성

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [17]:
from langchain.vectorstores import OpenSearchVectorSearch

In [18]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60    
)
vector_db

# 6. Chunking JSON Doc 

## Chunk Size and Chunk Overlap Size 결정

In [19]:
parent_chunk_size = 4096
parent_chunk_overlap = 0

child_chunk_size = 1024
child_chunk_overlap = 256

opensearch_parent_key_name = "parent_id"
opensearch_family_tree_key_name = "family_tree"


In [20]:
from local_utils.proc_docs import create_parent_chunk, create_child_chunk

## Parent Chunking

create_parent_chunk() 아래와 같은 작업을 합니다.
- all_docs 에 있는 문서를 parent_chunk_size 만큼으로 청킹 합니다.
- Parent Chunk 에 두개의 메타 데이타를 생성 합니다.
    - family_tree: parent
    - parent_id : None

In [21]:
parent_chunk_docs = create_parent_chunk(all_docs, opensearch_parent_key_name, 
                                        opensearch_family_tree_key_name,parent_chunk_size, parent_chunk_overlap)
print(f"Number of parent_chunk_docs= {len(parent_chunk_docs)}")


Number of parent_chunk_docs= 1948


In [22]:
parent_chunk_docs[0:1]

[Document(page_content='사용 방법 동영상.Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.이 섹션에는 Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.Knox E-FOTA 시작하기 이 동영상은 Knox E-FOTA 콘솔을 안내하며 리셀러 등록, 디바이스 승인, 캠페인 생성, 캠페인 할당, 디바이스 상태 모니터링 방법을 보여줍니다.Knox E-FOTA에서 캠페인 만들기 다음 동영상은 삼성 디바이스에 Knox E-FOTA 캠페인을 만들고 적용하는 방법에 대한 심층적인 정보를 제공합니다.Knox E-FOTA를 VMware Workspace ONE에 연결하기 다음 비디오에서는 Knox E-FOTA를 VMware Workspace ONE과 연결하는 동시에 워크스페이스 1에서 디바이스 그룹을 추가하는 간단한 단계를 설명합니다.', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 1, 'title': '사용 방법 동영상', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27', 'family_tree': 'parent', 'parent_id': None})]

OpenSearch 에 parent chunk 삽입

In [23]:
%%time

parent_ids = vector_db.add_documents(
                        documents = parent_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )



CPU times: user 4.44 s, sys: 259 ms, total: 4.7 s
Wall time: 4min 16s


In [24]:
total_count_docs = opensearch_utils.get_count(os_client, index_name)
print("total count docs: ", total_count_docs)


total count docs:  {'count': 1948, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


삽입된 Parent Chunk 의 첫번째를 확인 합니다. family_tree, parent_id 의 값을 확인 하세요.

In [25]:
def show_opensearch_doc_info(response):
    print("opensearch document id:" , response["_id"])
    print("family_tree:" , response["_source"]["metadata"]["family_tree"])
    print("parent document id:" , response["_source"]["metadata"]["parent_id"])
    print("parent document text: \n" , response["_source"]["text"])

response = opensearch_utils.get_document(os_client, doc_id = parent_ids[0], index_name = index_name)
show_opensearch_doc_info(response)    

opensearch document id: 34c32591-7d58-46e0-9665-a32975c2226c
family_tree: parent
parent document id: None
parent document text: 
 사용 방법 동영상.Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.이 섹션에는 Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.Knox E-FOTA 시작하기 이 동영상은 Knox E-FOTA 콘솔을 안내하며 리셀러 등록, 디바이스 승인, 캠페인 생성, 캠페인 할당, 디바이스 상태 모니터링 방법을 보여줍니다.Knox E-FOTA에서 캠페인 만들기 다음 동영상은 삼성 디바이스에 Knox E-FOTA 캠페인을 만들고 적용하는 방법에 대한 심층적인 정보를 제공합니다.Knox E-FOTA를 VMware Workspace ONE에 연결하기 다음 비디오에서는 Knox E-FOTA를 VMware Workspace ONE과 연결하는 동시에 워크스페이스 1에서 디바이스 그룹을 추가하는 간단한 단계를 설명합니다.


## Child Chunking

### Child Chunk 생성

아래의 create_child_chunk() 는 다음과 같은 작업을 합니다.
- parent_chunk_docs 각각에 대해서 Child Chunk 를 생성 합니다. 
- Child Chunk 에 두개의 메타 데이타를 생성 합니다.
    - family_tree: child
    - parent_id : parent 에 대한 OpenSearch document id

In [26]:
# child_chunk_docs = create_child_chunk(parent_chunk_docs[0:1], parent_ids)
child_chunk_docs = create_child_chunk(child_chunk_size, child_chunk_overlap, parent_chunk_docs, parent_ids, 
                                      opensearch_parent_key_name, opensearch_family_tree_key_name)
print(f"Number of child_chunk_docs= {len(child_chunk_docs)}")


Number of child_chunk_docs= 4025


### 생성된 Child 와 이에 대한 Parent 정보 확인

Child Chunk 한개에 대한 정보를 확인 합니다.

In [27]:
child_chunk_docs[0:1]

[Document(page_content='사용 방법 동영상.Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.이 섹션에는 Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.Knox E-FOTA 시작하기 이 동영상은 Knox E-FOTA 콘솔을 안내하며 리셀러 등록, 디바이스 승인, 캠페인 생성, 캠페인 할당, 디바이스 상태 모니터링 방법을 보여줍니다.Knox E-FOTA에서 캠페인 만들기 다음 동영상은 삼성 디바이스에 Knox E-FOTA 캠페인을 만들고 적용하는 방법에 대한 심층적인 정보를 제공합니다.Knox E-FOTA를 VMware Workspace ONE에 연결하기 다음 비디오에서는 Knox E-FOTA를 VMware Workspace ONE과 연결하는 동시에 워크스페이스 1에서 디바이스 그룹을 추가하는 간단한 단계를 설명합니다.', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 1, 'title': '사용 방법 동영상', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27', 'family_tree': 'child', 'parent_id': '34c32591-7d58-46e0-9665-a32975c2226c'})]

Child 에 대한 Parent 정보 확인

In [28]:
parent_id = child_chunk_docs[0].metadata["parent_id"]
print("child's parent_id: ", parent_id)
print("\n###### Search parent in OpenSearch")
response = opensearch_utils.get_document(os_client, doc_id = parent_id, index_name = index_name)
show_opensearch_doc_info(response)    


child's parent_id:  34c32591-7d58-46e0-9665-a32975c2226c

###### Search parent in OpenSearch
opensearch document id: 34c32591-7d58-46e0-9665-a32975c2226c
family_tree: parent
parent document id: None
parent document text: 
 사용 방법 동영상.Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.이 섹션에는 Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.Knox E-FOTA 시작하기 이 동영상은 Knox E-FOTA 콘솔을 안내하며 리셀러 등록, 디바이스 승인, 캠페인 생성, 캠페인 할당, 디바이스 상태 모니터링 방법을 보여줍니다.Knox E-FOTA에서 캠페인 만들기 다음 동영상은 삼성 디바이스에 Knox E-FOTA 캠페인을 만들고 적용하는 방법에 대한 심층적인 정보를 제공합니다.Knox E-FOTA를 VMware Workspace ONE에 연결하기 다음 비디오에서는 Knox E-FOTA를 VMware Workspace ONE과 연결하는 동시에 워크스페이스 1에서 디바이스 그룹을 추가하는 간단한 단계를 설명합니다.


### OpenSearch 에 Child 삽입

In [29]:
%%time

child_ids = vector_db.add_documents(
                        documents = child_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )

print("length of child_ids: ", len(child_ids))

length of child_ids:  4025
CPU times: user 9.04 s, sys: 497 ms, total: 9.54 s
Wall time: 7min 20s


In [30]:
total_count_docs = opensearch_utils.get_count(os_client, index_name)
print("total count docs: ", total_count_docs)


total count docs:  {'count': 5973, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


In [31]:
response = opensearch_utils.get_document(os_client, doc_id = child_ids[0], index_name = index_name)
show_opensearch_doc_info(response)    

opensearch document id: 655e2d71-2b8d-49bf-a995-739028e2b276
family_tree: child
parent document id: 34c32591-7d58-46e0-9665-a32975c2226c
parent document text: 
 사용 방법 동영상.Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.이 섹션에는 Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.Knox E-FOTA 시작하기 이 동영상은 Knox E-FOTA 콘솔을 안내하며 리셀러 등록, 디바이스 승인, 캠페인 생성, 캠페인 할당, 디바이스 상태 모니터링 방법을 보여줍니다.Knox E-FOTA에서 캠페인 만들기 다음 동영상은 삼성 디바이스에 Knox E-FOTA 캠페인을 만들고 적용하는 방법에 대한 심층적인 정보를 제공합니다.Knox E-FOTA를 VMware Workspace ONE에 연결하기 다음 비디오에서는 Knox E-FOTA를 VMware Workspace ONE과 연결하는 동시에 워크스페이스 1에서 디바이스 그룹을 추가하는 간단한 단계를 설명합니다.


# 7. 검색 테스트

## Lexical 검색

In [32]:
# q = "how to add image"
q = "이미지를 어떻게 추가해"
query ={'query': 
        {'bool': {'must': 
                  [{'match': 
                    {'text': 
                     {'query': "{q}", 'minimum_should_match': '0%', 'operator': 'or'}}}], 
                  'filter': {
                    "term": {
                      "metadata.family_tree": "child"
                    }                      
                  }
                 }
        }
       }
pprint(query)

{'query': {'bool': {'filter': {'term': {'metadata.family_tree': 'child'}},
                    'must': [{'match': {'text': {'minimum_should_match': '0%',
                                                 'operator': 'or',
                                                 'query': '{q}'}}}]}}}


In [33]:
# query = "how to add image"
# query = opensearch_utils.get_query(
#     query=query
# )

response = opensearch_utils.search_document(os_client, query, index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

There is no response


## 시맨틱 검색

In [34]:
vector_db.similarity_search(q, k=2)

[Document(page_content='사용자 지정 애니메이션 파일을 생성합니다.순서대로 재생할 이미지를 업로드하고 원하는 이미지 방향, 디더링 및 크기를 설정하여 사용자 지정 장치 부팅 애니메이션을 만들 수 있습니다.애니메이션을 생성하고 업로드한 후에는 프로필에 추가하고 장치에 할당하기 전에 애니메이션을 미리 보고 확인할 수 있습니다.애니메이션 만들기 맞춤 애니메이션을 만들려면: 1.왼쪽 탐색 메뉴에서 라이브러리를 클릭하고 미디어 탭을 클릭합니다. 2.오른쪽 상단에서 미디어 추가를 클릭합니다. 3.애니메이션 생성을 클릭하여 새 애니메이션 시퀀스를 생성합니다. 4.PNG 시퀀스 업로드 화면에서 애니메이션 프레임으로 사용할 최소 2개, 최대 99개의 PNG 이미지를 업로드합니다.업로드하는 PNG 파일의 크기 및 해상도가 대상 장치의 화면 크기 및 디스플레이 해상도와 동일해야 최적의 디스플레이가 보장됩니다.5.애니메이션에 이름을 지정하고 다음 설정을 구성합니다. 회전 - 애니메이션의 회전 각도를 선택합니다.크기 조정 - 애니메이션이 장치에 표시되는 방식을 선택합니다.디더링 - 파일 크기를 줄이는 데 사용되는 디더링 양을 지정합니다.설정이 높을수록 파일 크기는 줄어들지만 애니메이션 품질에는 영향을 미칠 수 있습니다. 6.완료되면 애니메이션 생성을 클릭하여 애니메이션을 생성합니다.애니메이션 업로드 기존 애니메이션을 QMG 파일로 업로드할 수도 있습니다.미디어 추가 화면에서 애니메이션 업로드를 선택하고 애니메이션 파일을 업로드합니다..QMG 형식의 파일만 지원됩니다.미디어 파일 삭제 라이브러리에서 미디어 파일을 삭제하려면: 1.왼쪽 탐색 메뉴에서 라이브러리를 클릭합니다. 2.현재 프로필에 사용되고 있지 않은 미디어 파일을 선택합니다. 3.오른쪽 상단에서 미디어 삭제를 클릭합니다. 4.파일 삭제를 클릭하여 파일 삭제를 확인합니다.', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 382, 'title': 

# 8. 검증 인덱스 생성

## Index 이름 결정

In [35]:
eval_index_name = "v26-genai-poc-knox-kor-eval-parent-doc-retriever"

## Sampling

In [36]:
import random
def get_sampling_doc(seed, ratio, docs):

    random.seed(seed)
    
    eval_docs = docs[:int(len(docs)*ratio)]
    
    return eval_docs
    
eval_docs = get_sampling_doc(seed=200, ratio=0.05, docs= all_docs)
print("eval docs: ", len(eval_docs))
eval_docs[0:2]
    
    

eval docs:  86


[Document(page_content='사용 방법 동영상.Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.이 섹션에는 Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.Knox E-FOTA 시작하기 이 동영상은 Knox E-FOTA 콘솔을 안내하며 리셀러 등록, 디바이스 승인, 캠페인 생성, 캠페인 할당, 디바이스 상태 모니터링 방법을 보여줍니다.Knox E-FOTA에서 캠페인 만들기 다음 동영상은 삼성 디바이스에 Knox E-FOTA 캠페인을 만들고 적용하는 방법에 대한 심층적인 정보를 제공합니다.Knox E-FOTA를 VMware Workspace ONE에 연결하기 다음 비디오에서는 Knox E-FOTA를 VMware Workspace ONE과 연결하는 동시에 워크스페이스 1에서 디바이스 그룹을 추가하는 간단한 단계를 설명합니다.', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 1, 'title': '사용 방법 동영상', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27'}),
 Document(page_content='녹스 E-FOTA.Knox E-FOTA를 사용하면 기업 IT 관리자가 사용자 개입 없이 OS 버전과 보안 업데이트를 기업 디바이스에 원격으로 배포할 수 있습니다.배포 전에 업데이트를 테스트하여 사내 앱과 새 OS 버전 간의 호환성을 확인하고, 최신 보안 패치를 일정에 따라 배포하여 엔터프라이즈 디바이스의 보안을 강화하세요.대상 고객 무료 체험하기 이 문서의 대상: 시스템 보안 설계자 — Knox E-FOTA의 작동 원리와 Knox E-FOTA를 사용하여 엔터프라이즈 디바이스를 업데이트하는 방법을 알아보세요.IT 관리자 - 엔터프라이즈 디바이스의 무선 업데이트를 관리하는 방법

## 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [37]:
index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    eval_index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        eval_index_name
    )
    
opensearch_utils.create_index(os_client, eval_index_name, index_body)
index_info = os_client.indices.get(index=eval_index_name)
print("Index is created")
pprint(index_info)    

index_name=v26-genai-poc-knox-kor-eval-parent-doc-retriever, exists=False

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'v26-genai-poc-knox-kor-eval-parent-doc-retriever'}
Index is created
{'v26-genai-poc-knox-kor-eval-parent-doc-retriever': {'aliases': {},
                                                      'mappings': {'properties': {'metadata': {'properties': {'last_updated': {'type': 'date'},
                                                                                                              'project': {'type': 'keyword'},
                                                                                                              'seq_num': {'type': 'long'},
                                                                                                              'source': {'type': 'keyword'},
                                                                                                              'title': {'type': 'text'},
       

## 검증 인덱스 연결 오브젝트 생성

In [38]:
eval_vector_db = OpenSearchVectorSearch(
    index_name= eval_index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60    
)
eval_vector_db

## Parent Chunking

In [39]:
parent_chunk_docs = create_parent_chunk(eval_docs, opensearch_parent_key_name, 
                                        opensearch_family_tree_key_name,parent_chunk_size, 
                                        parent_chunk_overlap)
print(f"Number of parent_chunk_docs= {len(parent_chunk_docs)}")


Number of parent_chunk_docs= 92


In [40]:
parent_chunk_docs[0:1]

[Document(page_content='사용 방법 동영상.Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.이 섹션에는 Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.Knox E-FOTA 시작하기 이 동영상은 Knox E-FOTA 콘솔을 안내하며 리셀러 등록, 디바이스 승인, 캠페인 생성, 캠페인 할당, 디바이스 상태 모니터링 방법을 보여줍니다.Knox E-FOTA에서 캠페인 만들기 다음 동영상은 삼성 디바이스에 Knox E-FOTA 캠페인을 만들고 적용하는 방법에 대한 심층적인 정보를 제공합니다.Knox E-FOTA를 VMware Workspace ONE에 연결하기 다음 비디오에서는 Knox E-FOTA를 VMware Workspace ONE과 연결하는 동시에 워크스페이스 1에서 디바이스 그룹을 추가하는 간단한 단계를 설명합니다.', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 1, 'title': '사용 방법 동영상', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27', 'family_tree': 'parent', 'parent_id': None})]

In [41]:
%%time

parent_ids = eval_vector_db.add_documents(
                        documents = parent_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )



CPU times: user 227 ms, sys: 16.2 ms, total: 243 ms
Wall time: 12.2 s


In [42]:
total_count_docs = opensearch_utils.get_count(os_client, eval_index_name)
print("total count docs: ", total_count_docs)


total count docs:  {'count': 92, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


In [43]:
response = opensearch_utils.get_document(os_client, doc_id = parent_ids[0], index_name = eval_index_name)
show_opensearch_doc_info(response)    

opensearch document id: d46ec1ef-61ed-4b4b-a3da-7cce336d2624
family_tree: parent
parent document id: None
parent document text: 
 사용 방법 동영상.Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.이 섹션에는 Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.Knox E-FOTA 시작하기 이 동영상은 Knox E-FOTA 콘솔을 안내하며 리셀러 등록, 디바이스 승인, 캠페인 생성, 캠페인 할당, 디바이스 상태 모니터링 방법을 보여줍니다.Knox E-FOTA에서 캠페인 만들기 다음 동영상은 삼성 디바이스에 Knox E-FOTA 캠페인을 만들고 적용하는 방법에 대한 심층적인 정보를 제공합니다.Knox E-FOTA를 VMware Workspace ONE에 연결하기 다음 비디오에서는 Knox E-FOTA를 VMware Workspace ONE과 연결하는 동시에 워크스페이스 1에서 디바이스 그룹을 추가하는 간단한 단계를 설명합니다.


## Child Chunking

In [44]:
# child_chunk_docs = create_child_chunk(parent_chunk_docs[0:1], parent_ids)
child_chunk_docs = create_child_chunk(child_chunk_size, child_chunk_overlap, parent_chunk_docs, 
                                      parent_ids, 
                                      opensearch_parent_key_name, opensearch_family_tree_key_name)
print(f"Number of child_chunk_docs= {len(child_chunk_docs)}")


Number of child_chunk_docs= 179


In [45]:
child_chunk_docs[0:1]

[Document(page_content='사용 방법 동영상.Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.이 섹션에는 Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.Knox E-FOTA 시작하기 이 동영상은 Knox E-FOTA 콘솔을 안내하며 리셀러 등록, 디바이스 승인, 캠페인 생성, 캠페인 할당, 디바이스 상태 모니터링 방법을 보여줍니다.Knox E-FOTA에서 캠페인 만들기 다음 동영상은 삼성 디바이스에 Knox E-FOTA 캠페인을 만들고 적용하는 방법에 대한 심층적인 정보를 제공합니다.Knox E-FOTA를 VMware Workspace ONE에 연결하기 다음 비디오에서는 Knox E-FOTA를 VMware Workspace ONE과 연결하는 동시에 워크스페이스 1에서 디바이스 그룹을 추가하는 간단한 단계를 설명합니다.', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 1, 'title': '사용 방법 동영상', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27', 'family_tree': 'child', 'parent_id': 'd46ec1ef-61ed-4b4b-a3da-7cce336d2624'})]

In [46]:
parent_id = child_chunk_docs[0].metadata["parent_id"]
print("child's parent_id: ", parent_id)
print("\n###### Search parent in OpenSearch")
response = opensearch_utils.get_document(os_client, doc_id = parent_id, index_name = eval_index_name)
show_opensearch_doc_info(response)    


child's parent_id:  d46ec1ef-61ed-4b4b-a3da-7cce336d2624

###### Search parent in OpenSearch
opensearch document id: d46ec1ef-61ed-4b4b-a3da-7cce336d2624
family_tree: parent
parent document id: None
parent document text: 
 사용 방법 동영상.Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.이 섹션에는 Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.Knox E-FOTA 시작하기 이 동영상은 Knox E-FOTA 콘솔을 안내하며 리셀러 등록, 디바이스 승인, 캠페인 생성, 캠페인 할당, 디바이스 상태 모니터링 방법을 보여줍니다.Knox E-FOTA에서 캠페인 만들기 다음 동영상은 삼성 디바이스에 Knox E-FOTA 캠페인을 만들고 적용하는 방법에 대한 심층적인 정보를 제공합니다.Knox E-FOTA를 VMware Workspace ONE에 연결하기 다음 비디오에서는 Knox E-FOTA를 VMware Workspace ONE과 연결하는 동시에 워크스페이스 1에서 디바이스 그룹을 추가하는 간단한 단계를 설명합니다.


In [47]:
%%time

child_ids = eval_vector_db.add_documents(
                        documents = child_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )

print("length of child_ids: ", len(child_ids))

length of child_ids:  179
CPU times: user 421 ms, sys: 13.9 ms, total: 435 ms
Wall time: 20 s


In [48]:
total_count_docs = opensearch_utils.get_count(os_client, eval_index_name)
print("total count docs: ", total_count_docs)


total count docs:  {'count': 271, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


In [49]:
response = opensearch_utils.get_document(os_client, doc_id = child_ids[5], index_name = eval_index_name)
show_opensearch_doc_info(response)    

opensearch document id: 4924e485-b2ef-4052-96fe-558061fd2717
family_tree: child
parent document id: 99f701cb-336a-4442-866c-550fa7642dc3
parent document text: 
 .강제 업데이트 (무음) ✔ ✔ ✔ 사용자 개입 없이 장치에 OS 업데이트를 배포할 수 있습니다.예약 업데이트 ✔ ✔ ✔ OS 업데이트를 다운로드하고 설치할 특정 날짜 및 시간 범위 (예: 업무 시간 외) 를 설정합니다.강제 업데이트 (중요) ✔ ✔ 중요한 작업이 진행 중인 동안 사용자가 업데이트를 연기할 수 있도록 허용합니다 (최대 지연 기간 포함).사용자는 업데이트를 거부할 수 없습니다.추가 캠페인 옵션 ✔ ✔ 다음 옵션 설정: 로밍 중 펌웨어 다운로드 허용.충전 도크에 연결된 경우에만 설치를 허용합니다.캠페인 상태 보기 기능 ✔ ✔ 캠페인 운영 상태 보기.디바이스 및 캠페인 상태에 대한 모니터링 대시보드 위젯은 Knox E-FOTA Advanced에서 사용할 수 있습니다. ✔ 대시보드를 통해 Knox E-FOTA의 다양한 영역을 모니터링할 수 있습니다.독립 웹 콘솔 ✔ ✔ 웹 포털을 통해 관리 작업을 수행합니다.네트워크 대역폭 제어 ✔ ✔ 설정된 최대 대역폭 내에서 펌웨어 업데이트를 배포합니다.Wi-Fi 전용 모드 ✔ ✔ Wi-Fi를 통해서만 다운로드 및 업데이트가 이루어지도록 설정하여 셀룰러 사용 비용을 절감할 수 있습니다.캠페인당 여러 모델 업데이트 가능 ✔ 단일 캠페인에서 여러 장치 모델을 서로 다른 펌웨어 릴리스에 할당할 수 있습니다.캠페인당 순차 업데이트 ✔ ✔ 단 하나의 캠페인으로 모든 버전에서 대상 버전으로 업데이트.즉, 기기를 현재 버전에서 대상 버전으로 점진적으로 업데이트하기 위해 여러 캠페인을 만들 필요가 없습니다.장치 그룹 및 조직 정보를 EMM과 동기화하는 기능 ✔ ✔ EMM에서 장치 그룹을 가져와서 장치 관리 작업을 간소화합니다.Knox E-FOTA

In [50]:
parent_id = response["_source"]["metadata"]["parent_id"]
print("child's parent_id: ", parent_id)
print("\n###### Search parent in OpenSearch")
response = opensearch_utils.get_document(os_client, doc_id = parent_id, index_name = eval_index_name)
show_opensearch_doc_info(response)    


child's parent_id:  99f701cb-336a-4442-866c-550fa7642dc3

###### Search parent in OpenSearch
opensearch document id: 99f701cb-336a-4442-866c-550fa7642dc3
family_tree: parent
parent document id: None
parent document text: 
 녹스 E-FOTA.Knox E-FOTA를 사용하면 기업 IT 관리자가 사용자 개입 없이 OS 버전과 보안 업데이트를 기업 디바이스에 원격으로 배포할 수 있습니다.배포 전에 업데이트를 테스트하여 사내 앱과 새 OS 버전 간의 호환성을 확인하고, 최신 보안 패치를 일정에 따라 배포하여 엔터프라이즈 디바이스의 보안을 강화하세요.대상 고객 무료 체험하기 이 문서의 대상: 시스템 보안 설계자 — Knox E-FOTA의 작동 원리와 Knox E-FOTA를 사용하여 엔터프라이즈 디바이스를 업데이트하는 방법을 알아보세요.IT 관리자 - 엔터프라이즈 디바이스의 무선 업데이트를 관리하는 방법을 알아보세요.솔루션을 사용해 보세요. Knox E-FOTA를 사용하여 모바일 업데이트 워크플로를 간소화하세요.버전 제어 관리를 활성화하고 업무 시간 외에 업데이트가 발생하도록 설정하여 OS 업데이트를 배포하여 생산성에 지장을 주지 않도록 하세요.튜토리얼 시작하기 Knox E-FOTA 소개 Knox E-FOTA"는 삼성 모바일 디바이스의 OS 버전을 제어하여 비용 효율성을 극대화하는 엔터프라이즈 솔루션입니다.IT 관리자는 이를 통해 다음과 같은 작업을 수행할 수 있습니다. 일정에 따라 최신 보안 패치를 디바이스에 배포하세요.배포 전에 업데이트를 테스트하여 사내 앱과 새 OS 버전 간의 호환성을 확인하십시오.주요 기능, 강제 업데이트 - 모든 장치에서 동일한 OS 버전을 사용할 수 있도록 강제 업데이트를 배포하여 장치 관리 환경을 간소화합니다.사용자 개입 없이 업데이트를 푸시할 수 있습니다.선택적 OS

## 검색 테스트

In [51]:
# q = "'how to use barcode"
q = "바코드를 어떻게 사용해"
query ={'query': 
        {'bool': {'must': 
                  [{'match': 
                    {'text': 
                     {'query': f"{q}", 'minimum_should_match': '0%', 'operator': 'or'}}}], 
                  'filter': {
                    "term": {
                      "metadata.family_tree": "child"
                    }                      
                  }
                 }
        }
       }
pprint(query)

{'query': {'bool': {'filter': {'term': {'metadata.family_tree': 'child'}},
                    'must': [{'match': {'text': {'minimum_should_match': '0%',
                                                 'operator': 'or',
                                                 'query': '바코드를 어떻게 사용해'}}}]}}}


In [52]:
response = opensearch_utils.search_document(os_client, query, eval_index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

# of searched docs:  10
# of display: 3
---------------------
_id in index:  7b9eb5c4-f3a9-405f-b889-3629d7a606a9
6.190286
캠페인 소개.녹스 E-FOTA 캠페인을 소개합니다.장치에 펌웨어 업데이트를 적용하려면 캠페인을 만들고 푸시할 펌웨어 버전을 지정해야 합니다.그런 다음 해당 캠페인에 기기를 할당합니다.캠페인을 활성화하면 펌웨어 업데이트가 할당된 장치에 푸시됩니다.캠페인이 활성화된 후 24시간마다 할당된 디바이스의 Knox E-FOTA 클라이언트 앱이 Knox E-FOTA 서버를 자동으로 폴링하여 캠페인에 적용된 정책 변경 사항을 확인합니다.Knox E-FOTA를 사용하면 두 가지 유형의 캠페인을 만들 수 있습니다. 일반 캠페인 - 디바이스에 프로덕션 펌웨어를 할당할 수 있음, 테스트 캠페인 - 디바이스에 더미 펌웨어를 할당할 수 있습니다. 이 섹션에서는 다양한 유형의 캠페인을 만드는 방법을 설명합니다.펌웨어 업데이트 중 디바이스가 어떻게 동작하는지에 대한 자세한 내용은 캠페인 중 디바이스는 어떻게 되나요? 를 참조하십시오.
{'source': 'all_processed_data_ko.json', 'seq_num': 31, 'title': '캠페인 소개', 'url': 'https://docs.samsungknox.com/admin/efota-one/features/create-a-campaign/campaign-intro', 'project': 'EFOTA', 'last_updated': '2023-07-26', 'family_tree': 'child', 'parent_id': 'da39910f-e571-496a-8692-f9fa7fd450d5'}
---------------------
_id in index:  1ffa4401-41b8-4e82-a3ef-d1c997705e9c
3.5985396
.사용자 개입 없이 업데이트를 푸시할 수 있습니다.선택적 OS"버전

In [53]:
eval_vector_db.similarity_search(q, k=2)

[Document(page_content='.장치에 태그 지정 태깅은 장치에 레이블을 지정하고 장치 목록에서 쉽게 검색하려는 경우에 유용합니다.예를 들어, 비즈니스 기능 또는 장치 사용자의 위치를 기준으로 장치에 레이블을 지정할 수 있습니다.이 섹션에서는 단일 장치에 태그를 지정하는 방법을 설명합니다.한 번에 여러 장치에 동일한 태그를 지정하려면 장치에서 일반적인 작업 수행을 참조하십시오. 1.Knox 관리 포털에 로그인하세요.왼쪽 사이드바에서 Knox E-FOTA를 클릭합니다. 2.디바이스 > 모든 디바이스로 이동합니다. 3.태그하려는 장치의 장치 ID를 클릭합니다. 4.장치 세부 정보 대화 상자의 태그 필드에 장치에 연결할 레이블을 입력합니다.예: 판매. 5.저장을 클릭합니다.', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 25, 'title': '디바이스 관리', 'url': 'https://docs.samsungknox.com/admin/efota-one/features/manage-devices', 'project': 'EFOTA', 'last_updated': '2023-09-06', 'family_tree': 'child', 'parent_id': 'acafba0e-e50c-4de4-bba4-4f61560d6e42'}),
 Document(page_content='. 3.추가를 클릭합니다.취소되거나 만료된 캠페인 취소되거나 만료된 캠페인 위젯은 종료된 시점부터 7일 동안 표시됩니다.각 기기 모델 및 기기 상태에 대한 성공률을 보여주는 원형 차트가 표시됩니다.그래프 위에 마우스를 올려 놓으면 자세한 내용을 볼 수 있습니다.취소되거나 만료된 캠페인 위젯이 여러 개 있을 수 있습니다.스케줄링된 캠페인 위젯은 곧 시작될 예정인 캠페인을 시작일별로 정렬하여 최대 5개까지 나열합니다.모두 보기를 클릭하면 예약된 캠페인만 보여주는 필터링된 캠페인 목록이 표시됩니다.이 위젯은 캠페인을 생성하자마자